<a href="https://colab.research.google.com/github/ItaloGomesCW/Analysis_Tool_Issuing/blob/main/Analysis_Tool_Issuing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing Libraries and Connecting to BigQuery

In [ ]:
#@title Importing Libraries
%%capture

#Manipulation
import pandas as pd
import numpy as np
import os
import re
import warnings
import time
import shutil
from datetime                     import datetime, timedelta
import datetime
from jinja2                       import BaseLoader, Environment


!pip install thefuzz
from thefuzz                      import fuzz, process

#Visualization
import plotly.offline as py
import plotly.express as px
import plotly.graph_objs as go
from IPython.display              import display, HTML, Markdown

#Colab and BigQuery
from google.colab                 import auth
from google.cloud                 import bigquery
from google.colab                 import syntax
from google.colab                 import data_table
from oauth2client.client          import GoogleCredentials


In [ ]:
#@title Authentication in BigQuery
try:
  auth.authenticate_user()
except:
  pass

os.environ["GCLOUD_PROJECT"] = "infinitepay-production"

client = bigquery.Client('infinitepay-production')

data_table.enable_dataframe_formatter()

In [ ]:
#@title Settings and Help Functions
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings("ignore")

def merchant_name(user_id):
  query = syntax.sql(f"""
  select merchant_name from maindb.merchants where user_id = {user_id}
  """)

  name = client.query(query).to_dataframe().iloc[0][0]
  return name.replace(" ", "_").replace("/", "_")

def phone_number_mask(number):
  if isinstance(number, str):
    if number.startswith("+55"):
      number = number.replace("+55", "")
      number = f"({number[0:2]}) {number[2:]}"
      number = f"{number[:len(number) - 4]}-{number[len(number) -4:]}"
  else: number
  return number

def concat_string(df):
  qtda_trx = ''
  for i in df.unique():
    if i != None:
      qtda_trx += f'{df.value_counts()[i]} {i}, '
  qtda_trx = qtda_trx[:-2]
  return qtda_trx

def func_color(val, color_if_true, color_if_false):
  color = color_if_true if val == 'approved' else color_if_false
  return 'background-color: {}'.format(color)

def func_color_notifications(val, color_if_true, color_if_false, search_text=['risk_monitoring_working_capital_warning',
                                                                              'risk_monitoring_identity_validated',
                                                                              'risk_monitoring_working_capital_second_warning',
                                                                              'risk_monitoring_identity_validation_request',
                                                                              'risk_id_validation_request_merchant',
                                                                              'block_merchant_notification',
                                                                              'termination_letter_settlement_blocked',
                                                                              'anomaly_transaction_doc_request',
                                                                              'chargeback_cdf_prevention_doc_request',
                                                                              'termination_letter',
                                                                              'risk_business_validation',
                                                                              'chargeback_fraud',
                                                                              'risk_monitoring']):
  color = color_if_true if val in search_text else color_if_false
  return 'background-color: {}'.format(color)

def func_color_offenses(val, search_text=['offense', 'suspicious', 'normal']):
  if val == 'offense':
    color = '#e44848'
    return 'background-color: {}'.format(color)
  elif val == 'suspicious':
    color = '#ffffb3'
    return 'color:black; background-color: {}'.format(color)
  elif val == 'normal':
    color = '#008000'
    return 'background-color: {}'.format(color)

###################### CONNECTIONS ######################
#As funções abaixo serão utilizadas na parte de Connected Merchant Information

max_graph_distance =  7
transactions_table = "maindb.transactions"

functions_subquery = """
#function to deduplicate simple array of any type
CREATE TEMP FUNCTION dedup(val ANY TYPE) AS ((
  SELECT ARRAY_AGG(distinct t)
  FROM UNNEST(val) t
));

#if duplicated merchant_entries inside array of struct, get the maximum amount
CREATE TEMP FUNCTION dedup_max_amount(val ANY TYPE) AS ((
    SELECT array_agg(STRUCT(merchant_id, status, approved_amount_24h, approved_count_24h))
    from
    (SELECT
        t.merchant_id,
        t.status,
        MAX(t.approved_amount_24h) approved_amount_24h,
        MAX(t.approved_count_24h) approved_count_24h,
    FROM UNNEST(val) t
    group by 1,2)
));

#function to sum values of the array of structs of connected merchants
CREATE TEMP FUNCTION sum_connected_merchant_struct_array(val ANY TYPE) AS ((
    SELECT STRUCT(approved_amount_24h, approved_count_24h)
    FROM(
    SELECT
        SUM(t.approved_amount_24h) as approved_amount_24h,
        SUM(t.approved_count_24h)  as approved_count_24h
    FROM UNNEST(val) t
    )
));

#sorts an array. Important to establish a partition key for merchants when looking at their transactions temporally getting
#the total 24h amount
CREATE temp function ascending_sort_array(val ANY TYPE) as ((
    SELECT
        ARRAY_AGG(t order by t)
    FROM unnest(val) t
));

#converts any type array to a string representation of it
CREATE temp function anytype_array_to_string(val ANY TYPE) as ((
    SELECT
        ARRAY_TO_STRING(ARRAY_AGG(CAST(t as STRING)), "#")
    from UNNEST(val) t
));


#Function to establish default statistics for a connection when connections
#didn't have an approved amount
CREATE TEMP FUNCTION default_connection_stats(val ANY TYPE) as ((
    select
        ARRAY_AGG(
            STRUCT(u.merchant_id as merchant_id,
                   u.status as status,
                   CAST(0 AS NUMERIC) as approved_amount_24h,
                   CAST(0 AS INT64) as approved_count_24h)
            )
    FROM UNNEST(val) u
));
CREATE TEMP FUNCTION normalized_text(x STRING)
    RETURNS STRING
    AS (LOWER(translate(x, "ŠŽšžŸÀÁÂÃÄÅÇÈÉÊËÌÍÎÏÐÑÒÓÔÕÖÙÚÛÜÝàáâãäåçèéêëìíîïðñòóôõöùúûüýÿ", "SZszYAAAAAACEEEEIIIIDNOOOOOUUUUYaaaaaaceeeeiiiidnooooouuuuyy")));

CREATE TEMP FUNCTION compare_names(x STRING, y STRING)
    RETURNS FLOAT64
    AS (SAFE_DIVIDE(
                   GREATEST(LENGTH(x), LENGTH(y)) - `ai-services-sae`.raw.levenshtein(normalized_text(x), normalized_text(y)),
                   GREATEST(LENGTH(x), LENGTH(y))
                   )
        );
"""
raw_connections_subquery = f"""
with markers as (
SELECT
    m.user_id AS merchant_id,
    l_r.name,
    SUBSTR(l_r.phone_number, -11) AS legal_rep_phone_number,
    --l_r.email,
    m.document_number,

FROM `infinitepay-production.maindb.merchants` AS m
LEFT JOIN `infinitepay-production.maindb.legal_representatives` AS l_r
    ON m.legal_representative_id = l_r.id
)

,aux_deliv_connections as (
    select
        CONCAT(
            CAST(cep AS STRING),
            "#",
            CAST(number AS STRING)
            ) as deliv_address,
        ARRAY_AGG(distinct merchant_id) as connected_merchants
    FROM maindb.delivery_addresses
    group by 1
)
,delivery_connections as (
    select
        merchant_id,
        a2.connected_merchants,
        -- a2.deliv_address

    from aux_deliv_connections a1, UNNEST(a1.connected_merchants) merchant_id
    left join aux_deliv_connections a2 on a1.deliv_address = a2.deliv_address
)

,aux_phone_number_connections as (
    select
        legal_rep_phone_number,
        ARRAY_AGG(merchant_id) connected_merchants,
    from markers
    where true
    and legal_rep_phone_number is not null
    and legal_rep_phone_number != "999999999"
    group by 1
)
,phone_number_connections as (
    select
        merchant_id,
        a2.connected_merchants

    from aux_phone_number_connections a1, UNNEST(a1.connected_merchants) merchant_id
    left join aux_phone_number_connections a2 on a1.legal_rep_phone_number = a2.legal_rep_phone_number
)
,aux_document_number_connections as (
    select
        document_number,
        ARRAY_AGG(merchant_id) connected_merchants,
    from markers
    where true
    and document_number is not null
    group by 1
)
,document_number_connections as (
    select
        merchant_id,
        a2.connected_merchants

    from aux_document_number_connections a1, UNNEST(a1.connected_merchants) merchant_id
    left join aux_document_number_connections a2 on a1.document_number = a2.document_number
)

,aux_pos_buyer_connections as (
    select
        card_token_id,
        ARRAY_AGG(distinct so.user_id IGNORE NULLS) connected_merchants,
    from maindb.transactions t
    left join maindb.sales_orders so on so.id = t.sales_order_id
    where true
    and capture_method='ecommerce' and merchant_id = 2625
    group by 1
)
,pos_buyer_connections as (
    select
        merchant_id,
        a2.connected_merchants,
    from aux_pos_buyer_connections a1, UNNEST(a1.connected_merchants) merchant_id
    left join aux_pos_buyer_connections a2 on a1.card_token_id = a2.card_token_id
)
,forced_connections as (
    SELECT
      user_id as merchant_id,
      ARRAY_AGG(user_id) as connected_merchants
    from maindb.merchants
    group by 1

)
"""

#Creating Functions and Queries

In [ ]:
#@title Offenses History
def offenses_history(user_id, name):
  display(Markdown(f'<H1>{name}'))
  display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Histórico de Offenses (Offenses History)<br>"))
  display(Markdown("<br>"))
  query = syntax.sql(f"""
    SELECT DISTINCT an.id,
                    u.email,
                    an.created_at,
                    an.description,
                    m.document_number,
                    an.automatic_pipeline,
                    analysis_type,
                    conclusion,
                    priority,
                    o.offense_group,
                    o.name,
    FROM `infinitepay-production.maindb.offense_analyses` an
      JOIN `infinitepay-production.maindb.offenses` as o ON o.id = an.offense_id
      LEFT JOIN `infinitepay-production.maindb.offense_actions` as act ON act.offense_analysis_id= an.id
      LEFT JOIN `infinitepay-production.maindb.merchants` as m ON m.user_id = an.user_id
      LEFT JOIN `infinitepay-production.maindb.users` u ON u.id = an.analyst_id
    WHERE TRUE AND an.user_id IN ({user_id})
    ORDER BY an.created_at DESC
  """)

  global df_offenses_history, merchant_nm
  merchant_nm = {name}
  df_offenses_history	= client.query(query).to_dataframe()
  df_offenses_history["description"]= df_offenses_history["description"].apply(lambda x: re.sub('Output.*?object','', x, flags=re.DOTALL))
  df_offenses_history["description"]= df_offenses_history["description"].apply(lambda x: re.sub('{"score.*?"} ','', x, flags=re.DOTALL))
  df_offenses_history["description"]= df_offenses_history["description"].apply(lambda x: re.sub('{"id.*?"} ','', x, flags=re.DOTALL))
  df_offenses_history['created_at'] = df_offenses_history['created_at'] - datetime.timedelta(hours=3)
  df_offenses_history_style = df_offenses_history.style.applymap(func_color_offenses, subset=['conclusion'])

  #display(HTML("<style>.container { width:100% !important; }</style>" + df_offenses_history_style.to_html()))
  display(df_offenses_history_style)

In [ ]:
#@title Merchant Notifications
def merchant_notifications(user_id):
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Notificações do Merchant (Merchant Notifications)"))
  display(Markdown("<br>"))

  notifications_query = syntax.sql(f"""
    SELECT  id,
            template,
            channels,
            created_at,
            status
    FROM maindb.notifications
    WHERE user_id={user_id}
    ORDER BY created_at DESC
  """)

  global result_analyses_notifications, df_notifications

  df_notifications = client.query(notifications_query).to_dataframe()
  df_notifications['created_at'] = df_notifications['created_at'] - datetime.timedelta(hours=3)
  df_notifications['date'] = df_notifications['created_at'].dt.date
  df_notifications['time'] = df_notifications['created_at'].dt.time.apply(lambda x: x.strftime('%H:%M:%S'))
  df_notifications = df_notifications.drop(['created_at'], axis=1)

  df_notifications_style = df_notifications.style.applymap(func_color_notifications, color_if_true='#e44848', color_if_false='#00800', subset=['template'])
  display(df_notifications_style)
  #display(HTML("<style>.container { width:100% !important; }</style>" + df_notifications_style.to_html()))


In [ ]:
#@title Merchant Information
def merchant_info(user_id):
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Informações do Merchant (Merchant Info)"))

  query = syntax.sql(f"""
    SELECT
      m.user_id,
      m.document_number,
      m.merchant_name,
      u.handle,
      u.email,
      u.phone_number,
      m.business_category,
      m.status as merchant_status,
      CASE
        WHEN DATE_DIFF(CURRENT_DATE(), DATE(m.opening_date), month) < 1 THEN CONCAT(DATE_DIFF(CURRENT_DATE(), DATE(m.opening_date), day), ' days')
        ELSE CONCAT(DATE_DIFF(current_date(), DATE(m.opening_date), month), ' months')
      END as opening_date,
      CASE
        WHEN DATE_DIFF(CURRENT_DATE(), DATE(m.created_at), month) < 1 THEN CONCAT(DATE_DIFF(CURRENT_DATE(), DATE(m.created_at), day), ' days')
        ELSE CONCAT(DATE_DIFF(current_date(), DATE(m.created_at), month), ' months')
      END as affiliation_date,
      m.created_at
      FROM maindb.merchants as m
      JOIN maindb.users as u ON u.id = m.user_id
      WHERE m.user_id = {user_id}
  """)

  lr_name = syntax.sql(f"""
    SELECT lr.name as lr_name, u.status as user_status FROM maindb.legal_representatives as lr
      JOIN maindb.merchants m ON m.legal_representative_id = lr.id
      JOIN maindb.users u ON u.id = m.user_id WHERE u.id = {user_id}
  """)

  withdraw = syntax.sql(f"""
    SELECT withdraw_locked FROM maindb.bank_accounts WHERE user_id = {user_id}
  """)

  pos_buyer = syntax.sql(f"""
    SELECT t.card_holder_name as who_bought_POS FROM maindb.sales_orders as s
      JOIN maindb.transactions as t ON t.sales_order_id = s.id
      WHERE s.user_id = {user_id}
      ORDER BY t.updated_at DESC limit 1

  """)

  validation = syntax.sql(f"""
    SELECT
      CONCAT(ARRAY_AGG(vendor ORDER BY created_at DESC)[offset(0)], " (", ARRAY_AGG(status ORDER BY created_at DESC)[offset(0)], ")") as validation,
      COUNT(*) as attempts,
      JSON_EXTRACT(ARRAY_AGG(response ORDER BY created_at DESC)[offset(0)], "$.Score") as score,
    FROM maindb.document_validations WHERE user_id = {user_id}
    GROUP BY user_id
  """)

  address = syntax.sql(f"""
    SELECT  cep,
            street,
            number,
            complement,
            neighborhood,
            CONCAT(city, "/" , state) as city,
            CONCAT(street, ", ", number, ", ", complement, ", ", neighborhood, ", ", city, "/" , state) as search_string
    FROM `maindb.addresses`
    WHERE user_id={user_id}
            """)

  delivery_address = syntax.sql("""
    SELECT  cep,
            street,
            number,
            complement,
            neighborhood,
            CONCAT(city, "/" , state) as city,
            CONCAT(street, ", ", number, ", ", complement, ", ", neighborhood, ", ", city, "/" , state) as search_string
    FROM `maindb.delivery_addresses`
    WHERE merchant_id={user_id}
            """.format(user_id=user_id))

#-------------------------------CRIANDO O DATAFRAME-------------------------------#

  global df_merchant_info, df_merchant_address, df_merchant_delivery_address

  df_merchant_info = client.query(query).to_dataframe()
  df_merchant_info = df_merchant_info.join(client.query(lr_name).to_dataframe())
  df_merchant_info = df_merchant_info.join(client.query(withdraw).to_dataframe())
  df_merchant_info = df_merchant_info.join(client.query(pos_buyer).to_dataframe())
  df_merchant_info = df_merchant_info.join(client.query(validation).to_dataframe())
  df_merchant_info["phone_number"] = df_merchant_info["phone_number"].apply(lambda x: phone_number_mask(x))

  df_merchant_address = client.query(address).to_dataframe()
  df_merchant_delivery_address = client.query(delivery_address).to_dataframe()

  df_merchant_info = df_merchant_info.transpose().rename(columns={0: "Informations"})
  df_merchant_address = df_merchant_address.transpose().rename(columns={0: "Address"})
  df_merchant_delivery_address = df_merchant_delivery_address.transpose().rename(columns={0: "Delivery Address"})
  df_addresses = pd.concat([df_merchant_address, df_merchant_delivery_address], axis=1)

  global result_analyses_merchant_info, var_merchant_name, var_document_number
  global LR_name, POS_name

  LR_name = df_merchant_info.loc["lr_name"]
  POS_name = df_merchant_info.loc["who_bought_POS"]
  var_merchant_name = df_merchant_info.loc["merchant_name"][0]
  var_document_number = df_merchant_info.loc["document_number"][0]

  times = f'{df_merchant_info.loc["affiliation_date"].values}, CNPJ {df_merchant_info.loc["opening_date"].values}'
  unico = f'UNICO score {df_merchant_info.loc["score"].values}' if pd.isna(df_merchant_info.loc['score'].values) != True else 'Not validated at UNICO'
  POS_buyer = (f'LR is the POS buyer.' if fuzz.token_set_ratio(df_merchant_info.loc["lr_name"], df_merchant_info.loc["who_bought_POS"]) > 75 else 'LR and POS buyer dont match.') if pd.isna(df_merchant_info.loc['who_bought_POS'].values) != True else 'Dont have POS.'

  result_analyses_merchant_info = f"Merchant {times}. {unico}. {POS_buyer}".replace("[", "").replace("]", "").replace("'", "")

  display(HTML("<style>.container { width:100% !important; }</style>" + df_merchant_info.to_html()))
  display(Markdown("<br>"))
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Endereços (Merchant Addresses)"))
  display(HTML("<style>.container { width:100% !important; }</style>" + df_addresses.to_html()))



In [ ]:
#@title Merchant Chargebacks
def chargebacks(user_id, interval1=30, interval2=60, interval3=90):
  display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Chargebacks do Merchant (Merchant Chargebacks)<br>"))
  display(Markdown("<br>"))

  cbk_query = syntax.sql(f"""
    SELECT  transaction_id,
            amount,
            created_at,
            status,
            reason,
            reason_code,
            source as source
    FROM maindb.chargebacks as cb
    WHERE merchant_id={user_id}
  """)

  billings_cbk_query = syntax.sql(f"""
    WITH amount_users AS (
        SELECT  tr.merchant_id AS user_id,
                SUM (IF (tr.status = 'approved', tr.amount, 0)) AS amount
        FROM maindb.transactions AS tr
        WHERE tr.merchant_id = {user_id}
        GROUP BY 1)
      -- ,billing_history
   SELECT
      w.user_id,
      au.amount,
      SUM(b.amount) - SUM(b.paid_amount) as pending_billing_amount,
      SAFE_DIVIDE(SUM(b.amount) - SUM(b.paid_amount),  au.amount ) as perc_pend_bill_trxapp
    FROM maindb.billings AS b
      JOIN maindb.wallets AS w ON w.id = b.wallet_id_sender
      JOIN maindb.chargeback_events ce ON ce.billing_id = b.id
      JOIN amount_users AS au ON au.user_id = w.user_id
    WHERE TRUE
    GROUP BY 1, 2
  """)

  global result_analyses_chargeback, df_cbk, df_chargebacks, df_billings_cbk

  df_cbk = client.query(cbk_query).to_dataframe()
  df_billings_cbk = client.query(billings_cbk_query).to_dataframe()
  #df_billings_cbk = round(df_billings_cbk['amount'].astype(float), 2)

  if len(df_cbk) > 0:
    data_30 = datetime.date.today() + datetime.timedelta(-30)
    data_60 = datetime.date.today() + datetime.timedelta(-60)
    data_90 = datetime.date.today() + datetime.timedelta(-90)

    df_cbk_30days = df_cbk.loc[df_cbk['created_at'] >= str(data_30)]
    df_cbk_60days = df_cbk.loc[df_cbk['created_at'] >= str(data_60)]
    df_cbk_90days = df_cbk.loc[df_cbk['created_at'] >= str(data_90)]

    df_cbk_sum = round(df_cbk['amount'].sum(), 2)
    df_cbk_sum_30days = round(df_cbk_30days['amount'].sum(), 2)
    df_cbk_sum_60days = round(df_cbk_60days['amount'].sum(), 2)
    df_cbk_sum_90days = round(df_cbk_90days['amount'].sum(), 2)

    df_cbk_count = len(df_cbk)
    df_cbk_count_30days = len(df_cbk_30days)
    df_cbk_count_60days = len(df_cbk_60days)
    df_cbk_count_90days = len(df_cbk_90days)

    dicio_chargebacks = {
      'Todo Período':[df_cbk_count, df_cbk_sum],
      'Últimos 30 dias':[df_cbk_count_30days, df_cbk_sum_30days],
      'Últimos 60 dias':[df_cbk_count_60days, df_cbk_sum_60days],
      'Últimos 90 dias':[df_cbk_count_90days, df_cbk_sum_90days]
      }

    result_analyses_chargeback = f" Merchant has {df_cbk_count} CBK for a total of R$ {df_cbk_sum}"
    df_chargebacks = pd.DataFrame(dicio_chargebacks).transpose().rename(columns={0: "Qtda", 1: "Somatório"})
    display(HTML("<style>.container { width:100% !important; }</style>" + df_chargebacks.to_html()))
    display(Markdown("<br>"))
    display(HTML("<style>.container { width:100% !important; }</style>" + df_cbk.to_html()))
    display(Markdown("<br>"))
    display(HTML("<style>.container { width:100% !important; }</style>" + df_billings_cbk.to_html()))
  else:
    result_analyses_chargeback = f"Merchant has no CBK."
    print(result_analyses_chargeback)

In [ ]:
#@title Merchant Transactions
def transactions(user_id, interval1=30, interval2=60, interval3=90):
  display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Transações do Merchant (Merchant Transactions)<br>"))
  display(Markdown("<br>"))

  query = syntax.sql(f"""
    SELECT  id,
            status,
            amount,
            installments,
            payment_method,
            capture_method,
            created_at,
            card_number,
            card_holder_name,
            card_brand,
            cardholder_id,
            card_token_id
    FROM maindb.transactions
    WHERE merchant_id = {user_id}
    ORDER BY created_at DESC
  """)

  global df_transactions, df_transactions_30days, df_transactions_60days, df_transactions_90days, df_transactions_scatter
  df_transactions = client.query(query).to_dataframe()

  if len(df_transactions) > 0:
    df_transactions['created_at'] = df_transactions['created_at'] - datetime.timedelta(hours=3)
    df_transactions['date'] = df_transactions['created_at'].dt.date
    df_transactions['time'] = df_transactions['created_at'].dt.time.apply(lambda x: x.strftime('%H:%M:%S'))
    df_transactions['amount'] = round(df_transactions['amount'].astype(float), 2)

    today = datetime.date.today()
    data_30 = today - datetime.timedelta(days=interval1)
    data_60 = today - datetime.timedelta(days=interval2)
    data_90 = today - datetime.timedelta(days=interval3)

    df_transactions_30days = df_transactions[df_transactions['date'] >= data_30.strftime('%Y-%m-%d')]
    df_transactions_60days = df_transactions[df_transactions['date'] >= data_60.strftime('%Y-%m-%d')]
    df_transactions_90days = df_transactions[df_transactions['date'] >= data_90.strftime('%Y-%m-%d')]

    '''data_30 = (datetime.date.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    data_60 = (datetime.date.today() - datetime.timedelta(days=60)).strftime('%Y-%m-%d')
    data_90 = (datetime.date.today() - datetime.timedelta(days=90)).strftime('%Y-%m-%d')

    df_transactions_30days = df_transactions[df_transactions['date'] >= data_30]
    df_transactions_60days = df_transactions[df_transactions['date'] >= data_60]
    df_transactions_90days = df_transactions[df_transactions['date'] >= data_90]'''

    df_transactions = df_transactions.drop(['created_at'], axis=1)

    df_transactions_sum = round(df_transactions.loc[df_transactions['status'] == 'approved']['amount'].sum(), 2)
    df_transactions_sum_30days = round(df_transactions_30days.loc[df_transactions_30days['status'] == 'approved']['amount'].sum(), 2)
    df_transactions_sum_60days = round(df_transactions_60days.loc[df_transactions_60days['status'] == 'approved']['amount'].sum(), 2)
    df_transactions_sum_90days = round(df_transactions_90days.loc[df_transactions_90days['status'] == 'approved']['amount'].sum(), 2)

    df_transactions_mean = round(df_transactions.loc[df_transactions['status'] == 'approved']['amount'].mean(), 2)
    df_transactions_mean_30days = round(df_transactions_30days.loc[df_transactions_30days['status'] == 'approved']['amount'].mean(), 2)
    df_transactions_mean_60days = round(df_transactions_60days.loc[df_transactions_60days['status'] == 'approved']['amount'].mean(), 2)
    df_transactions_mean_90days = round(df_transactions_90days.loc[df_transactions_90days['status'] == 'approved']['amount'].mean(), 2)

    df_transactions_median = round(df_transactions.loc[df_transactions['status'] == 'approved']['amount'].median(), 2)
    df_transactions_median_30days = round(df_transactions_30days.loc[df_transactions_30days['status'] == 'approved']['amount'].median(), 2)
    df_transactions_median_60days = round(df_transactions_60days.loc[df_transactions_60days['status'] == 'approved']['amount'].median(), 2)
    df_transactions_median_90days = round(df_transactions_90days.loc[df_transactions_90days['status'] == 'approved']['amount'].median(), 2)

    try:
      df_transactions_approved_rate = round(df_transactions['status'].value_counts()['approved']/len(df_transactions), 2) * 100
      df_transactions_approved_rate_30days = round(df_transactions_30days['status'].value_counts()['approved']/len(df_transactions_30days), 2) * 100
      df_transactions_approved_rate_60days = round(df_transactions_60days['status'].value_counts()['approved']/len(df_transactions_60days), 2) * 100
      df_transactions_approved_rate_90days = round(df_transactions_90days['status'].value_counts()['approved']/len(df_transactions_90days), 2) * 100
    except:
      df_transactions_approved_rate = f'0.00%'
      df_transactions_approved_rate_30days = f'0.00%'
      df_transactions_approved_rate_60days = f'0.00%'
      df_transactions_approved_rate_90days = f'0.00%'

    try:
      df_transactions_denied_rate = round(df_transactions['status'].value_counts()['denied']/len(df_transactions), 2) * 100
      df_transactions_denied_rate_30days = round(df_transactions_30days['status'].value_counts()['denied']/len(df_transactions_30days), 2) * 100
      df_transactions_denied_rate_60days = round(df_transactions_60days['status'].value_counts()['denied']/len(df_transactions_60days), 2) * 100
      df_transactions_denied_rate_90days = round(df_transactions_90days['status'].value_counts()['denied']/len(df_transactions_90days), 2) * 100
    except:
      df_transactions_denied_rate = f'0.00%'
      df_transactions_denied_rate_30days = f'0.00%'
      df_transactions_denied_rate_60days = f'0.00%'
      df_transactions_denied_rate_90days = f'0.00%'

  #else:
  #  df_transactions_sum, df_transactions_sum_30days, df_transactions_sum_60days, df_transactions_sum_90days = f'R$ 0.00', f'R$ 0.00', f'R$ 0.00', f'R$ 0.00'
  #  df_transactions_mean, df_transactions_mean_30days, df_transactions_mean_60days, df_transactions_mean_90days = f'R$ 0.00', f'R$ 0.00', f'R$ 0.00', f'R$ 0.00'
  #  df_transactions_median, df_transactions_median_30days, df_transactions_median_60days, df_transactions_median_90days = f'R$ 0.00', f'R$ 0.00', f'R$ 0.00', f'R$ 0.00'
  #  df_transactions_approved_rate, df_transactions_approved_rate_30days, df_transactions_approved_rate_60days, df_transactions_approved_rate_90days = f'0.00%', f'0.00%', f'0.00%', f'0.00%'
  #  df_transactions_denied_rate, df_transactions_denied_rate_30days, df_transactions_denied_rate_60days, df_transactions_denied_rate_90days = f'0.00%', f'0.00%', f'0.00%', f'0.00%'


  #-----------------------CNAE-----------------------#
  #QUERY PARA SELECIONAR AS INFORMAÇÕES DO CNAE
    cnae_query = syntax.sql(f"""
      #Selecionando o CNAE do merchant para salvar numa variável e usar nas querys para calcular as médias por CNAE
      SELECT cnae FROM maindb.merchants WHERE user_id = {user_id}
    """)

    #Salvando o CNAE numa variável
    cnae  = client.query(cnae_query).to_dataframe().iloc[0][0]

  #QUERY PARA CALCULAR A MÉDIA DE amount PELO CNAE
    average_cnae_query = syntax.sql(f"""
      SELECT average_cnae FROM (
        SELECT m.cnae, avg(t.amount) as average_cnae FROM maindb.merchants m
          JOIN maindb.transactions as t ON t.merchant_id = m.user_id
          WHERE t.status = "approved"
        GROUP BY 1) WHERE cnae = '{cnae}'
    """)

  #QUERY PARA CALCULAR A MÉDIA DE amount PELO cnae DOS ÚLTIMOS 30 DIAS
    average_cnae_query_30days = syntax.sql(f"""
      SELECT average_cnae_30days FROM (
        SELECT m.cnae, avg(t.amount) as average_cnae_30days FROM maindb.merchants as m
          JOIN maindb.transactions as t ON t.merchant_id = m.user_id
          WHERE t.status = "approved" AND t.created_at > DATE_ADD(CURRENT_DATE(), INTERVAL - 30 day)
        GROUP BY 1) WHERE cnae = '{cnae}'
    """)

  #QUERY PARA CALCULAR A MÉDIA DE amount PELO cnae DOS ÚLTIMOS 60 DIAS
    average_cnae_query_60days = syntax.sql(f"""
      SELECT average_cnae_60days FROM (
        SELECT m.cnae, avg(t.amount) as average_cnae_60days FROM maindb.merchants as m
          JOIN maindb.transactions as t ON t.merchant_id = m.user_id
          WHERE t.status = "approved" AND t.created_at > DATE_ADD(CURRENT_DATE(), INTERVAL - 60 day)
        GROUP BY 1) WHERE cnae = '{cnae}'
    """)

  #QUERY PARA CALCULAR A MÉDIA DE amount PELO cnae DOS ÚLTIMOS 90 DIAS
    average_cnae_query_90days = syntax.sql(f"""
      SELECT average_cnae_90days FROM (
        SELECT m.cnae, avg(t.amount) as average_cnae_90days FROM maindb.merchants as m
          JOIN maindb.transactions as t ON t.merchant_id = m.user_id
          WHERE t.status = "approved" AND t.created_at > DATE_ADD(CURRENT_DATE(), INTERVAL - 90 day)
        GROUP BY 1) WHERE cnae = '{cnae}'
    """)

    df_average_cnae_query = round(client.query(average_cnae_query).to_dataframe().iloc[0][0], 2)
    df_average_cnae_query_30days = round(client.query(average_cnae_query_30days).to_dataframe().iloc[0][0], 2)
    df_average_cnae_query_60days = round(client.query(average_cnae_query_60days).to_dataframe().iloc[0][0], 2)
    df_average_cnae_query_90days = round(client.query(average_cnae_query_90days).to_dataframe().iloc[0][0], 2)

    dicio_transactions = {
        'All Period':[df_transactions_sum, df_average_cnae_query, df_transactions_mean, df_transactions_median, df_transactions_approved_rate, df_transactions_denied_rate],
        'Last 30 days':[df_transactions_sum_30days, df_average_cnae_query_30days, df_transactions_mean_30days, df_transactions_median_30days, df_transactions_approved_rate_30days, df_transactions_denied_rate_30days],
        'Last 60 days':[df_transactions_sum_60days, df_average_cnae_query_60days, df_transactions_mean_60days, df_transactions_median_60days, df_transactions_approved_rate_60days, df_transactions_denied_rate_60days],
        'Last 90 days':[df_transactions_sum_90days, df_average_cnae_query_90days, df_transactions_mean_90days, df_transactions_median_90days, df_transactions_approved_rate_90days, df_transactions_denied_rate_90days]
        }

    global result_analyses_transactions, df_transactions_info
    result_analyses_transactions = f"Merchant has {len(df_transactions)} transactions ({concat_string(df_transactions['status'])}). From those, {concat_string(df_transactions['capture_method'])}"

    df_transactions_info = pd.DataFrame(dicio_transactions, index=['TPV', 'Média CNAE', 'Média amount', 'Mediana amount', '% approved', '% denied'])

    display(HTML("<style>.container { width:100% !important; }</style>" + df_transactions_info.to_html()))
    display(Markdown("<br>"))

    #-----------------------GRÁFICO BOXPLOT-----------------------#
    # Gerando gráficos das transações para Todo o Período
    trace1 = go.Box(y = df_transactions['amount'],
                    name = 'Todo o Período',
                    marker = {'color': '#f39c12'})
    # Gráfico gráficos das transações para os Últimos 30 dias
    trace2 = go.Box(y = df_transactions_30days['amount'],
                    name = 'Últimos 30 dias',
                    marker = {'color': '#e67e22'})
    # Gráfico gráficos das transações para os Últimos 60 dias
    trace3 = go.Box(y = df_transactions_60days['amount'],
                    name = 'Últimos 60 dias',
                    marker = {'color': '#d35400'})
    # Gráfico gráficos das transações para os Últimos 90 dias
    trace4 = go.Box(y = df_transactions_90days['amount'],
                    name = 'Últimos 90 dias',
                    marker = {'color': '#e74c3c'})
    data = [trace1, trace2, trace3, trace4]
    layout = go.Layout(title = 'BoxPlot das Transações em diferentes períodos (Pegando todos os status)',
                      titlefont = {'family': 'Arial',
                                    'size': 22,
                                    'color': '#7f7f7f'},
                      xaxis = {'title': 'Períodos'},
                      yaxis = {'title': 'Valor das Transações'},)
                      #paper_bgcolor = 'rgb(243, 243, 243)',
                      #plot_bgcolor = 'rgb(243, 243, 243)')
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
    #display(HTML("<style>.container { width:100% !important; }</style>" + py.iplot(fig).to_html()))

    df_transactions['LR_related'] = df_transactions['card_holder_name'].apply(lambda x: False if x == 'NAME/NO' else False if fuzz.token_set_ratio(LR_name, x) < 70 else True)
    df_transactions['POS_related'] = df_transactions['card_holder_name'].apply(lambda x: False if x == 'NAME/NO' else False if fuzz.token_set_ratio(POS_name, x) < 70 else True)
    df_transactions_scatter = df_transactions.copy()

    condicoes = [ (df_transactions_scatter['LR_related'] == True) | (df_transactions_scatter['POS_related'] == True),
                  (df_transactions_scatter['LR_related'] == False) | (df_transactions_scatter['POS_related'] == False)]
    opcoes = ['Is WC', 'Is not WC']
    df_transactions_scatter['Working_Capital'] = np.select(condicoes, opcoes)

    fig2 = px.scatter(df_transactions_scatter,
                  x="date",
                  y="amount",
                  color="status",
                  size='amount',
                  title='Dispersão das transações separadas por status',
                  hover_data=['id',
                              'status',
                              'amount',
                              'installments',
                              'payment_method',
                              'capture_method',
                              'date',
                              'time',
                              'LR_related',
                              'POS_related'],
                  color_discrete_map={"denied": "red",
                                      "approved": "green"})

    fig2.update_layout(
        title_font_family="Arial",
        title_font_color="#7f7f7f",
        title_font_size=22)
    py.iplot(fig2)

    fig3 = px.scatter(df_transactions_scatter,
                  x="date",
                  y="amount",
                  color="Working_Capital",
                  size='amount',
                  title='Dispersão das transações separadas por possíveis WC',
                  hover_data=['id',
                              'status',
                              'amount',
                              'installments',
                              'payment_method',
                              'capture_method',
                              'date',
                              'time',
                              'LR_related',
                              'POS_related'],
                  color_discrete_map={"Is not WC": "blue",
                                      "Is WC": "red"})
    fig3.update_layout(
        title_font_family="Arial",
        title_font_color="#7f7f7f",
        title_font_size=22)
    py.iplot(fig3)

    display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Todas as Transações<br>"))

    #df_transactions.columns = ['id',	'Status',	'Amount',	'Installments',	'Payment Method',	'Capture Method',	'Created at',	'Card Number',	'Card Holder Name',	'Bandeira',	'Card Holder id',	'Card Token id', 'LR Related', 'POS Related', 'Date', 'Time']
    df_transactions_style = df_transactions.copy()
    df_transactions_style = df_transactions_style[['id', 'status', 'amount', 'installments', 'payment_method', 'capture_method', 'date', 'time', 'card_number', 'card_holder_name',	'card_brand',	'cardholder_id', 'card_token_id', 'LR_related', 'POS_related']]
    df_transactions_style = df_transactions_style.style.background_gradient(cmap='coolwarm', subset=['amount', 'LR_related', 'POS_related']).format({'amount': '{:.2f}'}, precision=1)
    df_transactions_style = df_transactions_style.applymap(func_color, color_if_true='#008000', color_if_false='#e44848', subset=['status'])
    display(df_transactions_style)
  else:
    result_analyses_transactions = f"Merchant has no transactions."
    print(result_analyses_transactions)



def transactions_groupby(df, groupby_columns=['card_holder_name', 'card_number', 'card_token_id', 'card_brand', 'payment_method', 'capture_method', 'status']):
  df_transactions_groupby = df_transactions.groupby(groupby_columns, as_index=True).agg(amount_total=('amount', np.sum),
                                                                                                  amount_mean=('amount', np.mean),
                                                                                                  amount_max=('amount', np.max),
                                                                                                  amount_min=('amount', np.min))

  df_transactions_groupby = df_transactions_groupby.style.background_gradient(cmap='coolwarm',
                                                                                        subset=['amount_total', 'amount_mean', 'amount_max', 'amount_min']).format({'amount': '{:.2f}'}, precision=1)
  display(df_transactions_groupby)


In [ ]:
#@title CH Info for transaction_id
def ch_info_transaction_id(ch_transaction_id):
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Informações do CH pela transaction_id (CH Info for transaction_id)"))
  display(Markdown("<br>"))

  transactions_id_str = list(ch_transaction_id)
  transactions_id_str = ','.join(map(str, transactions_id_str))

  ch_info_transaction_id_query = syntax.sql(f"""
    SELECT  cardholder_user_id,
            cardholder_email,
            cardholder_phone,
            card_holder_name,
            card_number,
            card_token_id,
            human_id,
            human_cards_count,
            card_brand,
            issuer
    FROM `ai-services-sae.risk_ai.risk_analysis`
    WHERE transaction_id IN ({transactions_id_str})
    ORDER BY new_date DESC, new_time DESC
  """)

  global df_ch_info_transaction_id

  df_ch_info_transaction_id = client.query(ch_info_transaction_id_query).to_dataframe()

  display(df_ch_info_transaction_id)
  #display(HTML("<style>.container { width:100% !important; }</style>" + df_notifications_style.to_html()))


In [ ]:
#@title Card Holder Concentration
def ch_concentration(user_id, top=20):
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Concentração de CH (CH Concentration)"))
  display(Markdown("<br>"))

  cards_concentration_query = syntax.sql(f"""
    SELECT *,	SUM(b.percentage) OVER (ORDER BY b.percentage DESC) as acum
    FROM (SELECT
            CONCAT(t.card_holder_name,' - ',t.card_number) as card_name_number,
            #m.user_id,
            t.card_token_id,
            SUM(t.amount) as sum_trx,
            COUNT(t.id) as count_trx,
            ROUND(SAFE_DIVIDE(SUM(t.amount), (SELECT SUM(t.amount) FROM maindb.transactions as t WHERE t.status = 'approved' AND t.merchant_id = {user_id}))*100, 2) as percentage,
          FROM maindb.transactions as t
            JOIN maindb.merchants as m ON m.user_id = t.merchant_id
            WHERE m.user_id IN ({user_id}) AND t.status = 'approved'
            GROUP BY card_name_number, m.user_id, t.card_token_id
            ORDER BY 4 DESC) as b
   """)

  global df_cards_concentration, df_transactions_top20

  try:
    df_cards_concentration = client.query(cards_concentration_query).to_dataframe()
    cards_concentration = f'{df_cards_concentration["acum"].loc[4]}%'
  except:
    cards_concentration = '0.00%'

  df_transactions_top20 = df_cards_concentration.loc[0:top-1]
  df_transactions_top20.columns = ['CH name - Card number', 'Card Token ID', 'Total Transações', 'Qtda Transações', 'Perc.', 'Perc. Acumulado']
  df_transactions_top20[['Total Transações', 'Perc.', 'Perc. Acumulado']] = round(df_transactions_top20[['Total Transações', 'Perc.', 'Perc. Acumulado']].astype(float), 2)
  #display(df_transactions_top20)

  #display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Concentração de CH (Top 20)<br>"))
  display(HTML("<style>.container { width:100% !important; }</style>" + df_transactions_top20.to_html()))
  display(Markdown("<br>"))
  fig = px.histogram(df_transactions_top20,
                     x="CH name - Card number",
                     y='Total Transações',
                     title='CH concentration')
  fig.show()


In [ ]:
#@title Card Holder transacted on other Merchants
''' Depende da Merchant Information e Card Holder Concentration'''
def ch_other_merchant(card_token_id):
  display(Markdown("<H2 style='text-align:left;float:lfet;'>Verificar se os top CH transacionaram em outros Merchants"))
  display(Markdown("<br>"))

  card_token_id = tuple(card_token_id.dropna().unique())
  card_token_id = '(' + str(card_token_id[0]) + ')' if len(card_token_id) == 1  else card_token_id
  ch_other_merchant_query = syntax.sql(f"""
    SELECT  t.status,
            t.id,
            t.card_holder_name,
            t.card_token_id,
            t.card_number,
            t.amount,
            t.installments,
            DATE_SUB(t.created_at, interval 3 hour) as new_time,
            CONCAT(a.city,"-", a.state) as city,
            m.status as merchant_status,
            m.user_id,
            m.merchant_name,
            m.document_number,
            --pb.POS_buyer,
            b.withdraw_locked,
            m.business_category,
    FROM maindb.transactions as t
    JOIN maindb.merchants as m ON t.merchant_id=m.user_id
    JOIN maindb.bank_accounts as b ON b.user_id=m.user_id
    --LEFT JOIN pos_buyer pb ON pb.merchant_id = m.user_id
    JOIN maindb.addresses as a ON a.user_id=m.user_id

    WHERE t.card_token_id IN {card_token_id}
    ORDER BY t.created_at DESC
            """.format(card_token_id=card_token_id))

  global df_ch_other_merchant, result_analyses_ch_other_merchant
  result_analyses_ch_other_merchant = ''
  df_ch_other_merchant = client.query(ch_other_merchant_query).to_dataframe()
  df_ch_other_merchant = df_ch_other_merchant.loc[(df_ch_other_merchant['merchant_name'] != var_merchant_name) | (df_ch_other_merchant['document_number'] != var_document_number)].reset_index(drop=True)

  if len(df_ch_other_merchant) > 0:
    df_ch_other_merchant['amount'] = round(df_ch_other_merchant['amount'].astype(float), 2)
    df_ch_other_merchant['date'] = df_ch_other_merchant['new_time'].dt.date
    df_ch_other_merchant['time'] = df_ch_other_merchant['new_time'].dt.time.apply(lambda x: x.strftime('%H:%M:%S'))
    df_ch_other_merchant = df_ch_other_merchant.drop(['new_time'], axis=1)
    df_ch_other_merchant = df_ch_other_merchant[['status', 'id', 'card_number', 'card_holder_name', 'card_token_id', 'amount', 'installments', 'date', 'time', 'city', 'user_id', 'merchant_name', 'document_number', 'merchant_status', 'withdraw_locked', 'business_category']]
    df_ch_other_merchant

    display(HTML("<style>.container { width:100% !important; }</style>" + df_ch_other_merchant.to_html()))
  else:
    result_analyses_ch_other_merchant = 'CH did not transact in other Merchants'
    print(result_analyses_ch_other_merchant)

def ch_other_merchant_groupby(df, groupby_columns=['card_holder_name', 'merchant_name', 'merchant_status', 'city', 'withdraw_locked', 'business_category', 'status']):
  df_ch_other_merchant_groupby = df_ch_other_merchant.groupby(groupby_columns, as_index=True).agg(amount_total=('amount', np.sum),
                                                                                                  amount_mean=('amount', np.mean),
                                                                                                  amount_max=('amount', np.max),
                                                                                                  amount_min=('amount', np.min))

  df_ch_other_merchant_groupby = df_ch_other_merchant_groupby.style.background_gradient(cmap='coolwarm',
                                                                                        subset=['amount_total', 'amount_mean', 'amount_max', 'amount_min']).format({'amount': '{:.2f}'}, precision=1)
  display(df_ch_other_merchant_groupby)

In [ ]:
#@title Connected Merchant Information
def connected_information(user_id):
  display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Informações compartilhadas entre Merchants (Information shared between Merchants)<br>"))
  display(Markdown("<br>"))

  template = f"""
  {functions_subquery}
  {raw_connections_subquery}

  SELECT
      m.user_id as merchant_id,
      pos.connected_merchants pos_connections,
      doc.connected_merchants doc_connections,
      phone.connected_merchants phone_connections,
      deliv.connected_merchants delivery_connections,
  FROM maindb.merchants AS m
  LEFT JOIN pos_buyer_connections pos ON pos.merchant_id = m.user_id
  LEFT JOIN document_number_connections doc ON doc.merchant_id = m.user_id
  LEFT JOIN phone_number_connections phone ON phone.merchant_id = m.user_id
  LEFT JOIN delivery_connections deliv ON deliv.merchant_id = m.user_id
  WHERE
  m.user_id IN ({f"{user_id}"[1:-1]})
  """
  template = Environment(loader=BaseLoader()).from_string(source=template)
  template_context = {"max_graph_distance": max_graph_distance,
                      "transactions_table": transactions_table}

  global conn_df, df_pos_connections, df_doc_connections, df_phone_connections, df_delivery_connections

  conn_final = template.render(template_context)
  conn_df = client.query(conn_final).to_dataframe()
  display(HTML("<style>.container { width:100% !important; }</style>" + conn_df.to_html()))
  display(Markdown("<br>"))

  for j, i in enumerate(['pos_connections', 'doc_connections', 'phone_connections', 'delivery_connections']):
    j += 1
    if len(conn_df.iloc[:, j][0]) > 1:
      merchants_group = conn_df[i].iloc[0]
      variavel = [str(int) for int in merchants_group]
      variavel_query = ",".join(variavel)
      globals()['df_' + i] = client.query('''
        WITH pos_buyer AS (
          SELECT  m.status,
                  m.user_id AS merchant_id,
                  m.merchant_name,
                  m.document_number,
                  t.status,
                  t.card_holder_name AS POS_buyer,
                  t.card_number,
                  t.amount,
                  t.updated_at,
                  c.incoming_date,
                  c.source,
                  c.reason,
                  si.product_id,
                  p.description,
                  p.name
          FROM maindb.sales_orders AS s
          JOIN maindb.merchants as m ON m.user_id = s.user_id
          JOIN maindb.sales_items as si ON si.sales_order_id = s.id
          JOIN maindb.products as p ON p.id = si.product_id
          JOIN maindb.transactions as t ON t.sales_order_id = s.id
          LEFT JOIN maindb.chargebacks as c ON c.transaction_id = t.id
          WHERE 1=1 AND t.status <> 'denied' --and s.user_id in (623777)
          ORDER by t.updated_at DESC),

        --TPV
        total_sales AS (
          SELECT
              SUM(t.amount) AS sales,
              t.merchant_id AS id,
              COUNT(t.id) AS trx
          FROM maindb.transactions AS t
          JOIN maindb.merchants AS m ON m.user_id=t.merchant_id
          WHERE t.status = 'approved'
          GROUP BY t.merchant_id),

        blocked_merchants AS (
          SELECT m.user_id merchant, CONCAT(o.offense_group,'- ',o.name) AS reason
          FROM maindb.merchants m
          JOIN maindb.offense_analyses oa ON oa.user_id=m.user_id
          JOIN maindb.offenses o ON o.id=oa.offense_id
          WHERE TRUE AND m.status='blocked' AND oa.conclusion='offense' AND oa.priority='high')

        SELECT  m.status,
                ba.withdraw_locked,
                m.user_id,
                m.merchant_name,
                m.document_number,
                m.created_at,
                pb.POS_buyer,
                concat (lr.name, " ,", lr.cpf) AS legal_rep,
                IFNULL(ts.trx,0) AS trx_count,
                IFNULL(ts.sales,0) AS TPV,
                CONCAT(d.cep,",", d.number,",",d.city) AS address,
                CONCAT(da.cep,",", da.number,",",da.city) AS d_address,
                da.complement,
                bm.reason AS block_reason
        FROM maindb.merchants AS m
        LEFT JOIN pos_buyer AS pb ON pb.merchant_id=m.user_id
        JOIN view.legal_representatives AS lr ON lr.id=m.legal_representative_id
        LEFT JOIN total_sales ts ON ts.id=m.user_id
        LEFT JOIN maindb.bank_accounts AS ba ON ba.user_id=m.user_id
        LEFT JOIN maindb.addresses AS d ON d.user_id=m.user_id
        LEFT JOIN maindb.delivery_addresses AS da ON da.merchant_id=m.user_id
        LEFT JOIN blocked_merchants AS bm ON bm.merchant=m.user_id
        WHERE TRUE and m.user_id IN ({a})'''.format(a=variavel_query))

      globals()['df_' + i] = globals()['df_' + i].to_dataframe().drop_duplicates()
      globals()['df_' + i]['TPV'] = round( globals()['df_' + i]['TPV'].astype(float), 2)

      if i == 'pos_connections':
        display(Markdown("<H2 style='text-align:left;float:lfet;'>Combinação pela POS (March by POS)"))
      elif i == 'doc_connections':
        display(Markdown("<H2 style='text-align:left;float:lfet;'>Combinação pelo document_number (March by document_number)"))
      elif i == 'phone_connections':
        display(Markdown("<H2 style='text-align:left;float:lfet;'>Combinação pelo phone (March by phone)"))
      elif i == 'delivery_connections':
        display(Markdown("<H2 style='text-align:left;float:lfet;'>Combinação pelo address (March by address)"))

      display(Markdown("<br>"))
      display(HTML("<style>.container { width:100% !important; }</style>" + globals()['df_' + i].to_html()))


#Individual Analysis

In [ ]:
#@title Input the user_id
inicio = time.time()
user_id=8000819#@param

In [ ]:
offenses_history(user_id, merchant_name(user_id))

<H1>EMERSON_QUIRINO_BOTELHO

<br><H2 style='text-align:left;float:lfet;'>Histórico de Offenses (Offenses History)<br>

<br>

,id,email,created_at,description,document_number,automatic_pipeline,analysis_type,conclusion,priority,offense_group,name
0,4870787,bianca.oura@cloudwalk.io,2023-10-12 22:41:04.867000+00:00,Cardholder's user blocked because of accomplice_fraud,46438701840,True,automatic,offense,high,data_breach,carding
1,4859483,nikolas.cardoso@cloudwalk.io,2023-10-12 12:56:55.873000+00:00,Cardholder's user blocked because of accomplice_fraud,46438701840,True,automatic,offense,high,data_breach,carding
2,4857739,joao.dalaqua@cloudwalk.io,2023-10-12 11:48:35.307000+00:00,"Merchant has 162 trx in 2 days as CH, is part of fraudster human and its trying to fraud by carding.",46438701840,True,manual,offense,high,accomplice_fraud,carding
3,4809842,document_validation_user_alert@cloudwalk.io,2023-10-10 13:29:49.474000+00:00,User validated through unico platform.,46438701840,True,automatic,normal,low,unlock,account


In [ ]:
#merchant_notifications(user_id)

In [ ]:
merchant_info(user_id)

<H2 style='text-align:left;float:lfet;'>Informações do Merchant (Merchant Info)

,Informations
user_id,8000819
document_number,46438701840
merchant_name,EMERSON QUIRINO BOTELHO
handle,emerson_quirino_botelho_
email,emersonquirino79@gmail.com
phone_number,(13) 99739-2452
business_category,Restaurantes e similares
merchant_status,blocked
opening_date,465 months
affiliation_date,7 days


<br>

<H2 style='text-align:left;float:lfet;'>Endereços (Merchant Addresses)

,Address
cep,11770000
street,Rua Manuela Garcia de Souza
number,343
complement,casa
neighborhood,Jardim Veneza
city,ITARIRI/SP
search_string,"Rua Manuela Garcia de Souza , 343, casa, Jardim Veneza , ITARIRI/SP"


In [ ]:
transactions(user_id)

<br><H2 style='text-align:left;float:lfet;'>Transações do Merchant (Merchant Transactions)<br>

<br>

TypeError: ignored

In [ ]:
transactions_groupby(df_transactions, groupby_columns=['card_holder_name', 'card_number', 'card_token_id', 'card_brand', 'payment_method', 'capture_method', 'status'])

In [ ]:
ch_concentration(user_id, top=20)

In [ ]:
ch_other_merchant(df_cards_concentration['card_token_id'])

In [ ]:
ch_other_merchant_groupby(df_ch_other_merchant, groupby_columns=['card_holder_name', 'merchant_name', 'merchant_status', 'city', 'withdraw_locked', 'business_category', 'status'])

In [ ]:
chargebacks(user_id)

In [ ]:
#connected_information([user_id])

#Output

In [ ]:
display(Markdown(f'<H1>{user_id} - {merchant_nm}<br><br>'))
fim = time.time()
print(f'Tempo de execução {round(((fim - inicio)/60), 2)} Minutos')

display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Texto para Offense<br>"))
print(result_analyses_merchant_info + ' ' + result_analyses_chargeback)

display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Histórico de Offenses (Offenses History)<br>"))

display(Markdown("<H2 style='text-align:left;float:lfet;'>Notificações do Merchant (Merchant Notifications)<br>"))

display(Markdown("<H2 style='text-align:left;float:lfet;'>Informações do Merchant (Merchant Info)<br>"))
print(result_analyses_merchant_info)

display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Chargebacks do Merchant (Merchant Chargebacks)<br>"))
print(result_analyses_chargeback)

display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Transações do Merchant (Merchant Transactions)<br>"))
print(result_analyses_transactions)
print('')

display(Markdown("<H2 style='text-align:left;float:lfet;'>Concentração de CH (CH Concentration)<br><br>"))

display(Markdown("<H2 style='text-align:left;float:lfet;'>Verificar se os top CH transacionaram em outros Merchants<br>"))
print(result_analyses_ch_other_merchant)


display(Markdown("<br><H2 style='text-align:left;float:lfet;'>Informações compartilhadas entre Merchants (Information shared between Merchants)<br>"))
#display(result_analyses_merchant_info + result_analyses_transactions)




#Bugs para consertas e ideias para implementar


* Ta dando erro nesse EC pq não tem transação 2772085
* Inserir a colocaração do id transaction do CBK 160813
* <s>Inserir a tabela de notificações</s>
* 167061 Verificar os card_token_id == NaN
* Transactions, colocar o issuer, BIN e concentração de human
* <s>Verificar se os top CH transacionaram em outros merchants</s>
* Analisar o Merchant como CH
* adicionar table de related person
* <s>Merchant sem transação 484626 deu erro</s>
* <s>Consertar quando não tem transação 1791695</s>
* <s>Colab de CD não ta pegando a janela de pagamento certo 2093114</s>
* Não ta pegando a POS 2099848, 14207
* <s>VErificar o created_at do df de todas as transações</s>
* Implementar a tradução do BC
* Se o POS for diferente vê se ele tem relação ou transação com outros merchants
* Colocar ordem de análise por critério (quem tem dinheiro a receber e mais antigo)
* <s>Puxar os endereços </s>
* <s>Falso positivo LR e POS buyer quando não tem POS e o nome do cartão é NONE 25336</s>
* Pq não achou o comprador da POS 109264
* Verificar se o CBK ta pago
* Arrumar o months para year quando for mais que 12
* Inserir o texto da média do CNAE
* Inserir o texto da média das operações em relação ao CNAE
* Inserir o texto da concentração de CH
* <s>Inserir o texto do CBK</s>
* <s>Queri para verificar C6 BIN</s>
* Arrumar o df_transactions, quando tem card_token_id Null fica como decimal (23850)



In [ ]:
#def fun_teste(card_token_id):
#  ch_other_merchant_query = syntax.sql(f"""
'''    SELECT  t.status,
            t.id,
            t.card_holder_name,
            t.amount,
            t.installments,
            date_sub(t.created_at, interval 3 hour) as new_time,
            CONCAT (a.city,"-", a.state) as city,
            m.status as merchant_status,
            m.user_id,
            m.merchant_name,
            m.document_number,
            --pb.POS_buyer,
            b.withdraw_locked,
            m.business_category,
    FROM view.transactions as t
    JOIN view.merchants as m ON t.merchant_id=m.user_id
    JOIN view.bank_accounts as b ON b.user_id=m.user_id
    --LEFT JOIN pos_buyer pb ON pb.merchant_id = m.user_id
    JOIN view.addresses as a ON a.user_id=m.user_id

    WHERE t.card_token_id IN {card_token_id}
    ORDER BY t.created_at DESC'''
#            """.format(card_token_id=card_token_id))

#  teste = syntax.sql("""
#      SELECT  cep,
#            street,
#            number,
#            complement,
#            neighborhood,
#            CONCAT(city, "/" , state) as city,
#            CONCAT(street, ", ", number, ", ", complement, ", ", neighborhood) as search_string
#    FROM `view.delivery_addresses`
#    WHERE merchant_id={user_id}

#            """.format(user_id=user_id))

  #df_address	= client.query(ch_other_merchant_query).to_dataframe()
  #df_address = df_address.join(client.query(teste2).to_dataframe())
  #df_address = df_address.transpose().rename(columns={0: "Resultado"})
  #df_address = df_address.join(client.query(teste2).to_dataframe())

  #return df_address

In [ ]:
#fun_teste(tuple(df_cards_concentration_top5['card_token_id']))

In [ ]:
'''
    WITH amount_users AS
    (
      SELECT  tr.merchant_id AS user_id,
              sum (IF(tr.status = 'approved', tr.amount,0 )) AS amount
      FROM view.transactions tr
      GROUP BY 1
    )
    -- ,billing_history (
    SELECT
        w.user_id,
        au.amount,
        SUM(b.amount) - SUM(b.paid_amount) AS pending_billing_amount,
        SAFE_DIVIDE(SUM(b.amount) - SUM(b.paid_amount),  au.amount ) AS perc_pend_bill_trxapp
    FROM view.billings AS b
    JOIN view.wallets AS w on w.id = b.wallet_id_sender
    JOIN view.chargeback_events AS ce ON ce.billing_id = b.id
    JOIN amount_users au ON au.user_id = w.user_id
    WHERE TRUE
    GROUP BY 1, 2'''